In [35]:
import os
import json
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Parsing the Whole Folder

In [36]:
# Reading the whole folder

entities = []


# The index of these four arrays corresponds to one organization

orgs = []
texts = []
years = []
occurences = []


# The template counts the occurencies for each organization

years_template = { 2011: 0, 2012: 0, 2013: 0, 2014: 0, 2015: 0, 2016: 0, 2017: 0, 2018: 0, 2019: 0, 2020: 0 }


# Folder

dir = 'data/biomass/'
files = os.listdir(dir)
print('Total', len(files))

for index, filename in enumerate(files):

    # Read file
    
    f = open(dir + filename)
    r = f.read()
    data = json.loads(r)
    
    
    # Set basic metadata

    try:
        year = int(data[0]['publish_date'].split(' ')[0].split('-')[0]) # Set year
        records = data[0]['story_tags'] # Set tags
        if len(records) > 200: continue # Limit the maxiumum number of records
        
        if not index % 1000:
            print(index, ' | ', end=' ')
    
    except:
        continue


    # Collect entities
    
    tags = []

    for record in records:
        # If the record is an organization
        if (record['tag_set'] == 'cliff_organizations'):
            
            tag = record['tag']

            if len(tag) > 50:
                continue

            if len(tag) < 4:
                continue

            if tag[0].islower():
                continue # If the tag starts with lowercase

            if tag[0].isupper() and tag[1].isupper():
                continue # Remove acronyms
            
            tag = tag.replace('.', '')
            tag = tag.replace(',', '')
            tag = tag.replace(', Inc', '')
            tag = tag.replace(' Co', '')
            tag = tag.replace('US ', '')
            tag = tag.replace('mmission', ' Commission')
            tag = tag.replace('llege', ' College')
            tag = tag.replace('mmerce', ' Commerce')
            tag = tag.replace('lumbia', ' Columbia')
            tag = tag.replace('ourt', ' Court')
            tag = tag.replace('rporation', ' Corporation')
            tag = tag.replace('eurt', ' Coeurt')

            if tag == 'ExxonMobil': tag = 'Exxon'
            if tag == 'Exxon Mobil': tag = 'Exxon'
            if tag == 'Interior': tag = 'Interior Department'
            if tag == 'Reuters': tag = 'Thomson Reuters'
            if tag == 'Royal Dutch Shell': tag = 'Shell'
            if tag == 'World Health Organisation': tag = 'World Health Organization'
            if tag == 'International Monetary Found': tag = 'International Monetary Fund'
            if tag == 'European commission': tag = 'European Commission'
            if tag == 'Department of Agriculture': tag = 'Agriculture Department'
            if tag == 'Department of Energy': tag = 'Energy Department'
            if tag == 'State': tag = 'State Department'
            if tag == 'House': tag = 'White House'
            if tag == 'Sierra': tag = 'Sierra Club'
            
            if tag == 'Yale': tag = 'Yale University'
            if tag == 'Penn State': tag = 'Pennsylvania State University'
            if tag == 'Oxford of University': tag = 'University Oxford'
            if tag == 'Oxford': tag = 'Oxford University'
            if tag == 'Stanford': tag = 'Stanford University'
            if tag == 'Massachusetts Institute of Technology': tag = 'MIT'
            if tag == 'Harvard': tag = 'Harvard University'
            if tag == 'University of California, Berkeley': tag = 'University of California'

            if tag.startswith('Valens'): tag = 'Valens'
            if tag.startswith('Siemens'): tag = 'Siemens'
            if tag.startswith('Bloomberg'): tag = 'Bloomberg'
            if tag.startswith('Drax'): tag = 'Drax'

            if ('University' or 'College' or 'Institute') in tag:
                continue

            stoplist = {'Inc', 'Amazon rainforest', 'Agriculture', 'Energy', 'Ltd', 'Republican', 'MIT', 'Times', 'Parliament', 'UN', 'Liverpool'}
            
            if tag not in stoplist:
                tags.append(tag)
    
    
    # Create data structure

    for t in tags:

        related = tags.copy()
        related.remove(t)    
    
        if t not in orgs:
            orgs.append(t)
            i = orgs.index(t)
            texts.append(related)
            occurences.append(1)
            years.append(years_template.copy())
            years[i][year] += 1
                
        else:
            i = orgs.index(t)
            texts[i] += related
            occurences[i] += 1
            years[i][year] += 1

    # if index > 10:
    #     raise SystemExit("Stop right there!")

Total 26450
0  |  1000  |  2000  |  3000  |  4000  |  5000  |  6000  |  7000  |  8000  |  9000  |  10000  |  11000  |  12000  |  13000  |  14000  |  15000  |  16000  |  17000  |  18000  |  19000  |  20000  |  21000  |  22000  |  23000  |  24000  |  25000  |  26000  |  

## Copy

In [37]:
copy_orgs = orgs.copy()
copy_texts = texts.copy()
copy_years = years.copy()
copy_occurences = occurences.copy()

len(orgs)

32861

In [38]:
n = orgs.index('Bloomberg')

print(orgs[n], '\n')
# print(texts)

Bloomberg 



## CVS Export

In [39]:
import csv
pairs = []
with open('orgs.csv', 'w', encoding='UTF8') as f:
    for n in orgs:
        i = orgs.index(n)
        f.write(n + "," + str(len(n)) + ',' + str(occurences[i]) + '\n')

# Cleaning

In [40]:
orgs = copy_orgs.copy()
texts = copy_texts.copy()
years = copy_years.copy()
occurences = copy_occurences.copy()

# Remove less citates organizations

for index, occurrence in reversed(list(enumerate(occurences))):

    min = 40

    if occurrence < min:
        orgs.pop(index)
        texts.pop(index)
        years.pop(index)
        occurences.pop(index)
    # else:
        # print()
        # print(index, occurrence, orgs[index])
        # print()

# order years by key in an array of tuples

for index, y in enumerate(years):
    sortedDict = dict( sorted(y.items(), key=lambda x: x[0]) )
    _temp = {}
    for k,v in sortedDict.items():
        _temp[k] = v
    years[index] = _temp

print(len(orgs), len(texts), len(years), len(occurences))

# print(occurences[0])
# print(years[0][:, 0])

157 157 157 157


In [41]:
# Total linear regression

import numpy as np
from sklearn.linear_model import LinearRegression

total_years = {}

for year in years:
    # print(year)
    for k,v in year.items():
        # print(k, v)
        if k in total_years:
            total_years[k] += v
        else:
            total_years[k] = v

y = list(year.values())
x = list(year.keys())
x = np.array(x).reshape((-1, 1))

model = LinearRegression().fit(x, y)
slope = model.coef_
total_slope = slope[0]
# score = model.score(x, y)

total_years, total_slope


({2011: 826,
  2012: 789,
  2013: 1267,
  2014: 1586,
  2015: 2037,
  2016: 2498,
  2017: 1547,
  2018: 2896,
  2019: 3893,
  2020: 4639},
 1.0909090909090906)

In [42]:
# Linear regression

import matplotlib.colors

slopes = []
colors = []

_min = 0
_max = 0


# Slope

for year in years:

    y = list(year.values())
    x = list(year.keys())
    x = np.array(x).reshape((-1, 1))
    
    model = LinearRegression().fit(x, y)
    slope = model.coef_
    slope = slope[0] - total_slope
    # slope = slope[0]
    score = model.score(x, y)
    slopes.append(slope)

    if slope > _max: _max = slope
    if slope < _min: _min = slope

    # print()
    # print(list(year.keys()), y)
    # print('slope', slope, 'score', score)

print('min', _min, 'max', _max)

# Colors

cmap = plt.cm.RdYlBu_r
cmap = plt.cm.coolwarm
# norm = matplotlib.colors.Normalize(vmin=_min, vmax=_max)
norm = matplotlib.colors.DivergingNorm(vmin=_min, vcenter=0, vmax=_max)
# norm = matplotlib.colors.DivergingNorm(vmin=-10, vcenter=0, vmax=10)

for slope in slopes:
    color = cmap(norm(slope))
    colors.append(color)


min -1.8060606060606055 max 21.284848484848478


<ipython-input-42-50bf3f7e9c14>:41: MatplotlibDeprecationWarning: 
The DivergingNorm class was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use TwoSlopeNorm instead.
  norm = matplotlib.colors.DivergingNorm(vmin=_min, vcenter=0, vmax=_max)


In [43]:
# Term Frequency Matrix

import textacy

doc_term_matrix, dictionary = textacy.representations.build_doc_term_matrix(texts, tf_type="linear", idf_type="smooth")

In [48]:
# UMAP

import umap
from pointgrid import align_points_to_grid

# random_state=2
# reducer = umap.UMAP(n_components=2, n_neighbors=2, min_dist=0.001, metric='cosine')
reducer = umap.UMAP(n_components=2, n_neighbors=2, min_dist=0.1, metric='hellinger')

embedding = reducer.fit_transform(doc_term_matrix)
embedding = align_points_to_grid(embedding)
x = embedding[:, 0]; y = embedding[:, 1]

 * creating mesh with size 40 40
 * filling mesh


In [49]:
# Clustering on embedding

import hdbscan

# clusterer = hdbscan.HDBSCAN(min_cluster_size=4, min_samples=3, cluster_selection_epsilon=.5)
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, min_samples=2)
# clusterer = hdbscan.HDBSCAN(cluster_selection_epsilon=0.3, cluster_selection_method='leaf')
# min_samples is to consier all the elements that owtherwide will be classified as noise
# cluster_selection_epsilon extends clusters
clusterer.fit(embedding)
clusters = clusterer.labels_

# Grouping by cluster

values = set(clusters)
if -1 in values: values.remove(-1)

clusters = [[index for index, cluster in enumerate(clusters) if cluster==value] for value in values]

len(clusters)

13

In [50]:
# Plot

import math
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from scipy import interpolate




# Frame

plt.figure(figsize=(20,20), dpi=300)
plt.axis('off')


# Hulls

for cluster in clusters:

    # Average color
    
    background_color = []
    
    for i, index in enumerate(cluster):
        for occurence in range(occurences[index]):
            background_color.append([colors[index][0], colors[index][1], colors[index][2]])

    r = [i[0] for i in background_color]; r = sum(r) / len(r)
    g = [i[1] for i in background_color]; g = sum(g) / len(g)
    b = [i[2] for i in background_color]; b = sum(b) / len(b)

    background_color = (r, g, b, 1)

    # Hull

    points = []
    for index in cluster:
        points.append([embedding[index][0], embedding[index][1]])
    points = np.array(points)

    # print(points)

    hull = ConvexHull(points)
    
    x_hull = np.append(points[hull.vertices,0], points[hull.vertices,0][0]) # Collect the xs + first x
    y_hull = np.append(points[hull.vertices,1], points[hull.vertices,1][0])

    # print(x_hull)

    # break
    
    # interpolate
    dist = np.sqrt((x_hull[:-1] - x_hull[1:])**2 + (y_hull[:-1] - y_hull[1:])**2)
    dist_along = np.concatenate(([0], dist.cumsum()))
    spline, u = interpolate.splprep([x_hull, y_hull], u=dist_along, s=0)
    interp_d = np.linspace(dist_along[0], dist_along[-1], 50)
    interp_x, interp_y = interpolate.splev(interp_d, spline)
    
    # plot shape
    plt.fill(interp_x, interp_y, '--', c=background_color, alpha=.2)


# Scatterplot

# plt.scatter(x, y, s=occurences, c=colors)
plt.scatter(x, y, s=40, c=colors)


# Labels

for i, txt in enumerate(orgs):
    # text = plt.annotate(orgs[i], xy=(x[i], y[i] - math.sqrt(occurences[i]/math.pi)/40), ha='center', va='bottom')
    text = plt.annotate(orgs[i], xy=(x[i], y[i]), ha='center', va='bottom')
    text.set_fontsize(3)

plt.savefig('/Users/dario/Desktop/download.png')
plt.savefig('download.png')

    

